In [1]:
#RevImg is a lightweight package to interact with Google Reverse Image Search
#https://pypi.org/project/RevImg/
! pip install RevImg

You are using pip version 19.0, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [0]:
#https://www.google.com/searchbyimage?hl=en-US&image_url=https://a0.muscache.com/im/pictures/54335902/8572cc9d_original.jpg?aki_policy=xx_large&start=20

In [37]:
import requests
import json
from bs4 import BeautifulSoup

class RevImg:

	def __execute_query(self, img_url, count=0):
		base_google = "https://www.google.com/searchbyimage?hl=en-US&image_url="
		headers = {
		    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0',
		}
		url = "{}{}&start={}".format(base_google, img_url, count)
		r = requests.get(url, headers=headers, allow_redirects = True)
		return r.text

	def get_similar(self, img_url):

		q = self.__execute_query(img_url)
		soup = BeautifulSoup(q, 'html.parser')

		results = []

		for similar_image in soup.findAll('div', attrs={'rg_meta'}):
			tmp = json.loads(similar_image.get_text())
			img_url = tmp['ou']
			results.append(img_url)

		return results

	def get_best_guess(self, img_url):
		q = self.__execute_query(img_url)
		soup = BeautifulSoup(q, 'html.parser')
		for best_guess in soup.findAll('a', attrs={'class':'fKDtNb'}):
			result = best_guess.get_text()
		return result

	def get_related_results(self, img_url):
		count = 0
		keep_searching = True
		while keep_searching:
			q = self.__execute_query(img_url, count)
			soup = BeautifulSoup(q, 'html.parser')

			results = {
				'links': [],
				'descriptions': [],
			}
			for div in soup.findAll('div', attrs={'class':'rc'}):
				sLink = div.find('a')
				results['links'].append(sLink['href'])

			for desc in soup.findAll('span', attrs={'class':'st'}):
				results['descriptions'].append(desc.get_text())

			for link, description in zip(results["links"], results["descriptions"]):
				ret = {
					"link": link,
					"description": description
				}
				yield ret
			count += 10
			check_end_cur = soup.find("td", attrs={"class": "cur"})
			try:
				test = check_end_cur.next_sibling.td
			except AttributeError:
				keep_searching = False



In [44]:
class RevImg:

	def __execute_query(self, img_url, count=0):
		base_google = "https://www.google.com/searchbyimage?hl=en-US&image_url="
		headers = {
		    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0',
		}
		url = "{}{}&start={}".format(base_google, img_url, count)
		print(url)    
		r = requests.get(url, headers=headers, allow_redirects = True)
		return r.text
    
	def get_related_results_tst(self, img_url):
	    count = 0
	    q = self.__execute_query(img_url, count)
	    soup = BeautifulSoup(q, 'html.parser')

	    results = {
		    'links': [],
		    'descriptions': [],
	    }
	    for div in soup.findAll('div', attrs={'class':'rc'}):
	        sLink = div.find('a')
	        results['links'].append(sLink['href'])

	    for desc in soup.findAll('span', attrs={'class':'st'}):
	        results['descriptions'].append(desc.get_text())

	    for link, description in zip(results["links"], results["descriptions"]):
	        ret = {
	          "link": link,
	          "description": description
	        }
	        yield ret

In [46]:
#from RevImg import RevImg
ri = RevImg()

Image from Airbnb Listing:
https://www.airbnb.com/rooms/2311466?s=67&shared_item_type=1&virality_entry_point=1
<br>
<img src="https://a0.muscache.com/im/pictures/93e81544-5a5b-4b51-9d50-688b69b0660f.jpg?aki_policy=xx_large" width="400">

In [14]:
ri.get_related_results("https://a0.muscache.com/im/pictures/54335902/8572cc9d_original.jpg?aki_policy=xx_large")

AttributeError: 'RevImg' object has no attribute 'get_related_results'

In [39]:
#Evaluating Multiple Pages
#%%time
results = [ret for ret in ri.get_related_results("https://a0.muscache.com/im/pictures/93e81544-5a5b-4b51-9d50-688b69b0660f.jpg?aki_policy=xx_large")]

In [40]:
results

[{'link': 'https://www.ercol.com/contract/collections/originals/studio-couch/',
  'description': 'This versatile couch features an attractive steam bent frame consisting of tapered spindles and arched arm rests. The back rest is formed from a large board of\xa0...'},
 {'link': 'https://www.amazon.com/slp/studio-couch/t2tvaa9ek4gc9e9',
  'description': 'Buy products related to studio couch products and see what customers say about studio couch products on Amazon.com ✓ FREE DELIVERY possible on\xa0...'}]

In [47]:
#Evaluating One Page
#%%time
results = [ret for ret in ri.get_related_results_tst("https://a0.muscache.com/im/pictures/29a20538-f29f-45ee-932d-134d1a77312e.jpg?aki_policy=xx_large")]

https://www.google.com/searchbyimage?hl=en-US&image_url=https://a0.muscache.com/im/pictures/29a20538-f29f-45ee-932d-134d1a77312e.jpg?aki_policy=xx_large&start=0


In [35]:
results

[{'link': 'https://www.grove.co/',
  'description': 'Grove Collaborative delivers eco friendly home essentials, including household cleaning, personal care, baby, kid, and pet products. Shop trusted natural\xa0...'},
 {'link': 'https://en.wiktionary.org/wiki/grove',
  'description': 'From Middle English grove, grave, from Old English grāf, grāfa (“grove; copse”); compare English groove. Related to Old English grǣf, grǣfe (“brushwood; thicket\xa0...'},
 {'link': 'https://www.vrbo.com/en-ca/cottage-rental/p1407401vb',
  'description': '1024 × 768 - Cabin - C$85 avg/night - Tyler - Amenities include: Internet, Air Conditioning, TV, Satellite or Cable, Washer & Dryer, No Smoking, Heater ✓ Bedrooms: 1\xa0...'},
 {'link': 'https://www.homeaway.com/d/331980/tyler-state-park',
  'description': '360 × 270 - Book your Tyler State Park, Tyler vacation rentals online. Explore a large selection of vacation rentals, including Houses, Condos/Apartments & more: over 90\xa0...'},
 {'link': 'https://www

In [0]:
count = 0
for links in results:
  count = 1 + count
count

7

In [36]:
%%time
cross_listing_platforms = ['vrbo','homeaway','booking.com','travelocity','airbnb']
for links in results:
    
    if any(platform in links['link'] for platform in cross_listing_platforms):
        
        print (links['link'])

https://www.vrbo.com/en-ca/cottage-rental/p1407401vb
https://www.homeaway.com/d/331980/tyler-state-park
https://www.homeaway.com/d/106540/tiger-creek-wildlife-refuge
https://www.homeaway.com.sg/p1407401vb
https://www.vrbo.com/1407401
Wall time: 0 ns
